<h1 style='color: #6a5acd; font-size: 34px; font-weight: bold;'>Exploração Biblioteca UFRN

</h1>
#5abdcd

# <font color="#cda35a" style="font-size: 30px;">Objetivo</font>
<hr style="border: 2px solid #cda35a;">

 <p style='font-size: 23px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i><b> Explorar dos dados da biblioteca da UFRN, tais como _______ e indentificar possíveis problemas e soluções </b></i>     
</p>

---

# <font color="#cda35a" style="font-size: 30px;">Roteiro</font>
<hr style="border: 2px solid #cda35a;">

<h1 style='color: black; font-size:22px; font-weight: bold;'> 1. Importação de dados
</h1>

---

<h1 style='color: black; font-size: 22px; font-weight: bold;'>2. Limpeza de dados

</h1>

---

<h1 style='color: black; font-size: 22px; font-weight: bold;'> 
3. Análise exploratória de dados e DateTime
</h1>

---

<h1 style='color: black; font-size: 22px; font-weight: bold;'> 
4. Análise exploratória de dados e Variáveis categóricas
</h1>

---

<h1 style='color: black; font-size: 22px; font-weight: bold;'> 
5. Análise exploratória de dados e Boxplot
</h1>

---

<h1 style='color: black; font-size: 22px; font-weight: bold;'> 
6. JSON, Excel e Pivot_table
</h1>

---

<h1 style='color: black; font-size: 22px; font-weight: bold;'> 
7. Customização de tabelas
</h1>

---

# <font color="#cda35a" style="font-size: 30px;">1. Importação de dados</font>
<hr style="border: 2px solid #cda35a;">